<a href="https://colab.research.google.com/github/Franck-k3/Franck-k3/blob/main/NLP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IMPORTATION**

In [25]:
pip install rouge

In [26]:
import pandas as pd
from transformers import BertTokenizer, BertModel, pipeline
from sklearn.metrics.pairwise import cosine_similarity
from rouge import Rouge

#**Fonction pour obtenir l'embedding BERT d'un texte**

In [27]:
def get_bert_embedding(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

 # **Fonction pour comparer deux réponses d'un étudiant à un résumé généré à partir d'un prompt**

In [28]:

def compare_two_responses_with_summary(prompt_id, student_id1, student_id2):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')
    summarizer = pipeline("summarization")

    # Chargement des données
    prompts_df = pd.read_csv('prompts_train.csv')
    summaries_df = pd.read_csv('summaries_train.csv')

    # Trouver le texte du prompt et générer un résumé
    prompt_text = prompts_df[prompts_df['prompt_id'] == prompt_id]['prompt_text'].values[0]
    prompt_summary = summarizer(prompt_text, max_length=130, min_length=30, do_sample=False)[0]["summary_text"]

    # Trouver les réponses des étudiants correspondant au prompt_id donné
    response1 = summaries_df[(summaries_df['prompt_id'] == prompt_id) & (summaries_df['student_id'] == student_id1)]['text'].values[0]
    response2 = summaries_df[(summaries_df['prompt_id'] == prompt_id) & (summaries_df['student_id'] == student_id2)]['text'].values[0]

    # Calculer les embeddings BERT pour les réponses et le résumé du prompt
    response1_embedding = get_bert_embedding(response1, tokenizer, model)
    response2_embedding = get_bert_embedding(response2, tokenizer, model)
    prompt_summary_embedding = get_bert_embedding(prompt_summary, tokenizer, model)

    # Calculer la similarité cosinus pour chaque réponse avec le résumé du prompt
    similarity1 = cosine_similarity(response1_embedding, prompt_summary_embedding)
    similarity2 = cosine_similarity(response2_embedding, prompt_summary_embedding)

    return similarity1[0][0], similarity2[0][0]


# **Exemple d'utilisation**

In [29]:

prompt_id = "814d6b"  # Remplacez avec le prompt_id approprié
student_id1 = "000e8c3c7ddb"  # Remplacez avec l'ID de l'étudiant
student_id2 = "0070c9e7af47"
similarite1, similarite2 = compare_two_responses_with_summary(prompt_id, student_id1, student_id2)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Affichage

In [30]:
print(f"Similarité entre la première réponse de l'étudiant et le résumé du prompt : {similarite1}")
print(f"Similarité entre la seconde réponse de l'étudiant et le résumé du prompt : {similarite2}")

Similarité entre la première réponse de l'étudiant et le résumé du prompt : 0.707939863204956
Similarité entre la seconde réponse de l'étudiant et le résumé du prompt : 0.8359315395355225


# Fonction pour comparer la concaténation de deux réponses avec un résumé généré du prompt

In [31]:
def compare_concatenated_responses_with_summary(prompt_id, student_id1, student_id2):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')
    summarizer = pipeline("summarization")

    # Chargement des données
    prompts_df = pd.read_csv('prompts_train.csv')
    summaries_df = pd.read_csv('summaries_train.csv')

    # Trouver le texte du prompt et générer un résumé
    prompt_text = prompts_df[prompts_df['prompt_id'] == prompt_id]['prompt_text'].values[0]
    prompt_summary = summarizer(prompt_text, max_length=130, min_length=30, do_sample=False)[0]["summary_text"]

    # Trouver les réponses des étudiants correspondant au prompt_id donné
    response1 = summaries_df[(summaries_df['prompt_id'] == prompt_id) & (summaries_df['student_id'] == student_id1)]['text'].values[0]
    response2 = summaries_df[(summaries_df['prompt_id'] == prompt_id) & (summaries_df['student_id'] == student_id2)]['text'].values[0]

    # Concaténer les réponses
    concatenated_response = response1 + " " + response2

    # Calculer les embeddings BERT pour la réponse concaténée et le résumé du prompt
    concatenated_response_embedding = get_bert_embedding(concatenated_response, tokenizer, model)
    prompt_summary_embedding = get_bert_embedding(prompt_summary, tokenizer, model)

    # Calculer la similarité cosinus
    similarity = cosine_similarity(concatenated_response_embedding, prompt_summary_embedding)

    return similarity[0][0]

# Exemple d'utilisation
prompt_id = "814d6b"  # Remplacez avec le prompt_id approprié
student_id1 = "000e8c3c7ddb"  # Remplacez avec l'ID de la première réponse de l'étudiant
student_id2 = "0070c9e7af47"  # Remplacez avec l'ID de la seconde réponse de l'étudiant

similarite_concat = compare_concatenated_responses_with_summary(prompt_id, student_id1, student_id2)
print(f"Similarité entre la réponse concaténée des étudiants et le résumé du prompt : {similarite_concat}")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Similarité entre la réponse concaténée des étudiants et le résumé du prompt : 0.8116200566291809


In [33]:
def evaluate_summary_performance(model_summary, reference_summary):
    rouge = Rouge()
    scores = rouge.get_scores(model_summary, reference_summary)

    return scores


model_summary = "Votre résumé généré"
reference_summary = "Résumé de référence"
scores = evaluate_summary_performance(model_summary, reference_summary)
print(scores)

[{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}}]
